## Huggung face models

Testing some models for potential comparison to other models.

In [1]:
import open_clip
from metaspace import SMInstance
import torch

from ionimage_embedding.dataloader.clr_data import CLRdata

/home/trose/.conda/envs/torch-gpuCUDA12/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
open_clip.version.__version__

'2.22.0'

In [4]:
model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')
tokenizer = open_clip.get_tokenizer('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')

In [3]:
# Download some images:
ds_list = [
    '2022-12-07_02h13m50s',
    '2023-05-23_20h44m02s',
    '2023-09-29_10h49m52s'
                  ]

clrdat = CLRdata(ds_list, test=0.3, val=0.1, cache=False, cache_folder='/scratch/model_testing')

100%|████████████████████████████████████████| 980/980 [00:03<00:00, 304.24it/s]


(903, 470, 470)
1
2
3


In [ ]:
clrdata.full_dataset.images.shape